# **Minecraft Server on Google Colab!**
---

Start a server
> The script below will start your server. You'll have to create a server first to be able to use it.

In [ ]:
java_version = "25"
server_directory = "minecraft-server"
server_memory = "6G"
jar_name = "server.jar"
server_flags = "-XX:+UseG1GC -XX:+ParallelRefProcEnabled -XX:MaxGCPauseMillis=200 -XX:+UnlockExperimentalVMOptions -XX:+DisableExplicitGC -XX:+AlwaysPreTouch -XX:G1NewSizePercent=30 -XX:G1MaxNewSizePercent=40 -XX:G1HeapRegionSize=8M -XX:G1ReservePercent=20 -XX:G1HeapWastePercent=5 -XX:G1MixedGCCountTarget=4 -XX:InitiatingHeapOccupancyPercent=15 -XX:G1MixedGCLiveThresholdPercent=90 -XX:G1RSetUpdatingPauseTimePercent=5 -XX:SurvivorRatio=32 -XX:+PerfDisableSharedMem -XX:MaxTenuringThreshold=1 -Dusing.aikars.flags=https://mcflags.emc.gs -Daikars.new.flags=true"
memory_allocation = "-Xms" + server_memory + " -Xmx" + server_memory
# Choose tunnel service
# Available Options: ngrok, playit, none
tunnel_service = "playit"
# Ngrok token
authtoken = "" # Set Ngrok token every start or replace it
# Ngrok region
region = "na" # Change this to region you want

def check_java():
  java = !java -version 2>&1 | awk -F[\"\.] -v OFS=. 'NR==1{print $2}'
  return java[0]

def remove_default_java():
  !sudo apt remove -y openjdk-11-jdk-headless openjdk-11-jre-headless &>/dev/null && echo "java 11 removed correctly" || echo "error removing java 11"
  !sudo apt autoremove -y &>/dev/null && echo "packages cleaned correctly" || echo "error cleaning packages"

def install_java(version):
  !sudo apt update &>/dev/null && echo "apt updated correctly" || echo "error updating apt"
  !sudo apt install -y wget apt-transport-https &>/dev/null && echo "installed wget and apt-transport-https" || echo "error installing wget and apt-transport-https"
  !mkdir -p /etc/apt/keyrings &>/dev/null && echo "created key directory" || echo "error creating key directory"
  !wget -O - https://packages.adoptium.net/artifactory/api/gpg/key/public | tee /etc/apt/keyrings/adoptium.asc &>/dev/null && echo "obtaining keys" || echo "error obtaining keys"
  !echo "deb [signed-by=/etc/apt/keyrings/adoptium.asc] https://packages.adoptium.net/artifactory/deb $(awk -F= '/^VERSION_CODENAME/{print$2}' /etc/os-release) main" | tee /etc/apt/sources.list.d/adoptium.list &>/dev/null && echo "keys added" || echo "error adding keys"
  !sudo apt update &>/dev/null && echo "apt updated correctly" || echo "error updating apt"
  !sudo apt install temurin-{version}-jdk &>/dev/null && echo "java {version} installed correctly." || echo "error installing java {version}."
  !sudo apt clean &>/dev/null && echo "packages cleaned correctly" || echo "error cleaning packages"

def remove_java(version):
  !sudo apt remove -y temurin-{version}-jdk &>/dev/null && echo "java {version} removed correctly" || echo "error removing java {version}"
  !sudo apt autoremove -y &>/dev/null && echo "packages cleaned correctly" || echo "error cleaning packages"

def start_server():
  try:

    if check_java() == "11":
      remove_default_java()
      install_java(java_version)

    if check_java() != java_version:
      remove_java(check_java())
      install_java(java_version)

    if check_java() != java_version: raise ValueError("java " + java_version + " is not installed or is not working")

    print("java " + java_version + " is working correctly, you can continue.")
    from google.colab import drive

    drive.mount("/content/drive")
    %cd "/content/drive/My Drive/{server_directory}"
    !ls
    print("Using", tunnel_service)

    if tunnel_service == "ngrok":
      !pip -q install pyngrok
      from pyngrok import conf, ngrok

      print("Obtain your token from https://dashboard.ngrok.com/get-started/your-authtoken")
      import getpass
      if authtoken == "": authtoken = getpass.getpass()
      !ngrok authtoken $authtoken
      conf.get_default().region = region

      url = ngrok.connect(25565, 'tcp')
      print("Your server direction is " + ((str(url).split('"')[1::2])[0]).replace("tcp://", ""))
      print("Starting server...")
      !java $memory_allocation $server_flags -jar $jar_name nogui

    elif tunnel_service == "playit":
      !curl -SsL https://playit-cloud.github.io/ppa/key.gpg | gpg --dearmor | sudo tee /etc/apt/trusted.gpg.d/playit.gpg >/dev/null
      !echo "deb [signed-by=/etc/apt/trusted.gpg.d/playit.gpg] https://playit-cloud.github.io/ppa/data ./" | sudo tee /etc/apt/sources.list.d/playit-cloud.list
      !sudo apt update &>/dev/null && sudo apt install playit &>/dev/null && echo "Playit.gg installed" || echo "error installing Playit.gg"
      print("Starting server...")
      !playit & java $memory_allocation $server_flags -jar $jar_name nogui

    elif tunnel_service == "none":
      print("Starting server...")
      !java $memory_allocation $server_flags -jar $jar_name nogui

    else: raise ValueError("Tunnel service not valid")

  except ValueError as e:
    print(e)

start_server()

Make a server
> The script below will download a server and accept the EULA. Then you can start it.

In [ ]:
# Replace "1.21.8" with version you want.
# Available versions:
# - 1.21.8
# - 1.21.7
# - 1.21.6
# - 1.21.5
# - 1.21.4
# - 1.21
# - 1.20.6
# - 1.20.4
# - 1.20.2
# - 1.20.1
# - 1.20
# - 1.19.2
# - 1.19.1
# - 1.19
# - 1.18.2
# - 1.18.1
# - 1.18
# - 1.17.1
# - 1.17
# - 1.16.4
# - 1.16.3
# - 1.15.2
# - 1.14.4
# - 1.13.2
# - 1.12.2
# - 1.11.2
# - 1.10.2
# - 1.9.4
# - 1.8.8
# New versions maybe work, but is not guaranteed.
version = "1.21.8"
# Choose server type. Available versions are paper, purpur, divinemc, fabric, folia, vanilla and spigot
server_type = "folia"
jar_name = "server.jar"
server_directory = "minecraft-server"

def make_server():
  try:

    from google.colab import drive

    drive.mount("/content/drive")
    !mkdir "/content/drive/My Drive/{server_directory}"
    %cd "/content/drive/My Drive/{server_directory}"

    import requests

    if server_type == "paper":
      a = requests.get("https://api.papermc.io/v2/projects/paper/versions/" + version)
      b = requests.get("https://api.papermc.io/v2/projects/paper/versions/" + version + "/builds/" + str(a.json()["builds"][-1]))
      url = "https://api.papermc.io/v2/projects/paper/versions/" + version + "/builds/" + str(a.json()["builds"][-1]) + "/downloads/" + b.json()["downloads"]["application"]["name"]

    elif server_type == "purpur":
      url = "https://api.purpurmc.org/v2/purpur/" + version + "/latest/download"

    elif server_type == "divinemc":
      a = requests.get("https://mcjars.app/api/v1/builds/DIVINEMC/" + version + "/latest?tracking=none")
      url = str(a.json()["build"]["jarUrl"])

    elif server_type == "fabric":
      url = "https://meta.fabricmc.net/v2/versions/loader/" + version + "/0.17.2/1.1.0/server/jar"

    elif server_type == "folia":
      a = requests.get("https://mcjars.app/api/v1/builds/FOLIA/" + version + "/latest?tracking=none")
      url = str(a.json()["build"]["jarUrl"])

    elif server_type == "vanilla":
      a = requests.get("https://mcjars.app/api/v1/builds/VANILLA/" + version + "/latest?tracking=none")
      url = str(a.json()["build"]["jarUrl"])

    elif server_type == "spigot":
      a = requests.get("https://mcjars.app/api/v1/builds/SPIGOT/" + version + "/latest?tracking=none")
      url = str(a.json()["build"]["jarUrl"])

    print("Downloading to Google Drive...")
    r = requests.get(url)
    if r.status_code == 200:
      with open("/content/drive/My Drive/" + server_directory + "/" + jar_name, "wb") as f:
        f.write(r.content)
    else: raise ValueError("Error " + str(r.status_code) + "! It seems that you choose an unsupported version. Try running the script again.")

    !echo "eula=true" >> eula.txt
    print("Server is ready!")

  except ValueError as e:
    print(e)

make_server()